# SentinelOne Investigate Notebook

This notebook provides easy access to the SentinelOne API to investigate a single endpoint.

## Requirements
- Python3
- Python modules: Numpy, Pandas, SentinelOne-Management-SDK

## Setup
You'll need to download the SentinelOne Management SDK from the `Console > ? > API Doc` page. Just follow the setup instructions for the SDK.

```
python3 -m pip install numpy pandas
python3 -m pip install sentinel-mgmt-sdk.tar.gz
```

## Initialization
Now you'll need to grab your API token from the `My User > Options` link at the top right of your SentinelOne console. Store this somewhere safe, it is only shown to you once. Now continue by filling in your api_token and console host 
below and clicking run.

In [ ]:
import json
from management.mgmtsdk_v2_1.mgmt import Management
from management.mgmtsdk_v2_1.services.threat import ThreatQueryFilter
from management.mgmtsdk_v2_1.services.agent import AgentQueryFilter
try:
    import numpy as np
    import pandas as pd
    from pandas.io.json import json_normalize
except:
    print("Error importing Numpy or Pandas")


console_host = "whatever.sentinelone.net"
api_token = ""
sentinel = Management(hostname=console_host, api_token=api_token)

# Investigate

## Lookup Agent Information
Just update the `hostname` variable with a partial match, the query filter on line 5 uses the CONTAIN operator.


In [ ]:
hostname = "SAMPLECOMPUTER"
operator = "contains" # eq, like, contains

filter = AgentQueryFilter()
filter.apply('computerName', hostname, op=operator)
endpoints = sentinel.agents.get(query_filter=filter)
ep_json = endpoints.json['data']
ep_table = pd.DataFrame(ep_json)
ep_table[['computerName','siteName','agentVersion','scanStatus','scanFinishedAt','threatRebootRequired','userActionsNeeded']]

## Threats by Agent Name
Just update the `hostname` variable below, optionally changing the `operator` to adjust the query results.

In [ ]:
hostname = "SAMPLECOMPUTER"
operator = "contains" # eq, like, contains

filter = ThreatQueryFilter()
filter.apply('computerName', hostname, op=operator)
endpoint = sentinel.threats.get(query_filter=filter)
ep_json = endpoint.json['data']
ep_table = pd.DataFrame(ep_json)
ep_table[['agentDetectionInfo']]

## Scan an Agent(s)
Initiate endpoint scans by full or partial match query, just update the `hostname` and `operator` for your query. Be very careful with `like` and `contains` queries as they may match a large number of endpoints.

In [ ]:
hostname = "SAMPLECOMPUTER"
operator = "eq" # eq, like, contains

filter = AgentQueryFilter()
filter.apply('computerName', hostname, op=operator)
endpoints = sentinel.agent_actions.initiate_scan(query_filter=filter)
endpoints.json['data']

## Disconnect Agents
Disconnect endpoints from the network with a full or partial match query. Again, be careful as you may target a bunch of endpoints.

In [ ]:
hostname = "SAMPLECOMPUTER"
operator = "eq" # eq, like, contains

filter = AgentQueryFilter()
filter.apply('computerName', hostname, op=operator)
endpoints = sentinel.agent_actions.disconnect_from_network(query_filter=filter)
endpoints.json['data']

## Fetch Endpoint Logs
Initiate log collection by full or partial match query, just update the `hostname` and `operator` for your query and optionally adjust which logs to collect. Default Collects both Sentinel eventlogs as well as Windows forensic data and logs.

* Be very careful with `like` and `contains` queries as they may match a large number of endpoints.
* Logs can be downloaded from the `Console > Activity` page.

In [ ]:
hostname = "SAMPLECOMPUTER"
operator = "eq" # eq, like, contains
windows_logs = True
sentinel_logs = False

filter = AgentQueryFilter()
filter.apply('computerName', hostname, op=operator)
endpoints = sentinel.agent_actions.fetch_logs(agent_logs=sentinel_logs, platform_logs=windows_logs, query_filter=filter)